# Intermediate Chatbot

# Initial Setup

In [1]:
import os
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# Local bot
OLLAMA_API_KEY = "ollama"
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama_api = OpenAI(api_key=OLLAMA_API_KEY, base_url=OLLAMA_BASE_URL)
ollama_model = "llama3.2"

# Online bot
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "type-your-api-key-here")
gpt_api = OpenAI()
gpt_model = "gpt-4o-mini"

# Prompting

Adding facts, tone/style, and example to system prompt.

In [3]:
system_message = "You are a very helpful and friendly assistant in the Kampus Merdeka program. You encourage students to apply for the program. \
Especially  Magang MSIB and Studi Independen. For example, if a student wants to apply, \
you can reply something like, 'Great - we have a variety of programs - including some that might fit your educational background'. \
Assist and encourage the student to apply if they are unsure which program to take. \
If the student's college is under the Kementerian Agama, politely explain that they cannot apply."

In [4]:
def chat(message, history):
    """Gradio dedicated function for chatbot"""

    updated_system_message = system_message

    # Nano version of RAG
    if "link" in message:
        if not "link" in updated_system_message:
            updated_system_message += "If you are asked for a link to apply a program, direct the student to create an account first and apply here https://kampusmerdeka.kemdikbud.go.id/."
    
    messages = [{"role": "system", "content": updated_system_message}] + history + [{"role": "user", "content": message}]

    stream = ollama_api.chat.completions.create(
        model=ollama_model,
        messages=messages,
        stream=True,
        temperature=0.7
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

# UI

In [5]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
